In [41]:
import pandas as pd
import numpy as np
import netCDF4
from netCDF4 import Dataset
import os
from pyproj import Proj
from datetime import datetime
import datetime
import sys
import time
from netCDF4 import num2date, date2num
import h5py
import IPython
import time
import timeit
from IPython.display import display, HTML
import os
import glob
import sys
import sys
from pygeotile.tile import Tile
from pygeotile.point import Point
import tarfile
from six.moves import urllib
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import networkx as ntx
from scipy import ndimage as nd
from skimage import io

# from bokeh.io import output_file, output_notebook, show
# from bokeh.models import (
#   GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
#     DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
# )
# from bokeh.models.mappers import ColorMapper, LinearColorMapper
# from bokeh.palettes import Viridis5


# Put your path here
#Quadtree_level = 21
csv_path = r'D:\Buston\Boston2017_18.csv'
UL = (-71.163736,   42.458042 )
BR = (-71.006377,   42.278300)

spatial_interval = 1000  # meter
time_interval = 60*24     # miniute
mid_point = ((UL[0] + BR[0])/2, (UL[1]+ BR[1])/2)
UTM_zone = int((mid_point[0])/6) + 30
if mid_point[0] > 0:   # needs to be verified.
    UTM_zone +=1


# proj = cimgt.MapQuestOSM()
# plt.figure(figsize=(20,20), dpi=200)
# ax = plt.axes(projection=proj.crs)


def GetAllfiles(path, ext):
    cmd = path + '\*.' + ext
    all_files = glob.glob(cmd)
    return all_files


print("mid_point: ", mid_point)

sphere = 'north'
if  mid_point[1] < 0:
    sphere = 'south'

myProj = Proj("+proj=utm +zone={}, +{}+ellps=WGS84 +datum=WGS84 +units=m +no_defs".format(UTM_zone, sphere))
UTMx, UTMy = myProj(mid_point[0], mid_point[1])
print(UTMx, UTMy )
lon, lat = myProj(UTMx + spatial_interval, UTMy + spatial_interval, inverse=True)
resolu_x = lon - mid_point[0]
resolu_y = lat - mid_point[1]
cols = int((BR[0] - UL[0])/resolu_x)+1
rows = int((UL[1] - BR[1])/resolu_y)+1
print("row, col: ", rows, cols)

print("resolu_x, resolu_y: ", resolu_x, resolu_y)

#%time print(User_cnr_cube[:,:,:].sum())

def Add_value2HDF5(file_path, sGroup, cube):
    print("Start to append HDF5...")
    with h5py.File(path_name, 'a') as outfile:
        #dset = outfile.create_dataset('S_T_cube', data=cube, chunks=True, compression="gzip")
        #outfile.create_group('Data Fields')
        #dset = outfile.create_dataset('sGroup', data=cube, chunks=True, compression="gzip", shuffle=False)
        #dset = outfile.create_dataset('Data_Fields/User_Cnt', data=userCnt_cube, chunks=(rows, cols, 1), compression="gzip", shuffle=False)
        dset.attrs['resolu_x'] = resolu_x
        dset.attrs['resolu_y'] = resolu_y
        dset.attrs['UL_x'] = UL[0]
        dset.attrs['UL_y'] = UL[1]
        dset.attrs['Day_user_cnt'] = cube
    print("Append HDF5 finished.")

def AppendDailyFiles(df, date_field, path):
    df[date_field] = pd.to_datetime(df[date_field])
    df = df.set_index(date_field)
    df = df.sort_index()
    df['date'] = df.index
    dates = df.resample('D').index.strftime('%Y%m%d')
    for date in dates:
        global start_date
        start_date = pd.to_datetime(date, format='%Y%m%d')
        twts_eachday = df.loc[date]
        #print('twts_eachday: ', twts_eachday)
        #hours = twts_eachday.resample('H').size()
        #hours = twts_eachday.resample('H').index.hour    
        #twts_cube, user_cube = Fill_Twts_Users_Cnt_Cube(twts_eachday, resolu_x, resolu_y, UL[0], UL[1])
        print(date)
        Add_value2HDF5( twts_cube, user_cube, path + '\\' + date + '.hdf')
    
    return df    

def GetRowCol(res_x, res_y, x, y, ori_x, ori_y):  # start from zero
    #print('x, y: ', x, y)
    row = int((x - ori_x)/res_x)
    #print('row: ', row)
    col = int(abs(y - ori_y)/res_y)
    #print('col: ', col)
    return row, col

def ReadTweetsInRectangle(csv_path,UL, BR):
    tweets = pd.read_csv(csv_path, encoding="utf-8-sig")
    tweets_clip = tweets[
        (tweets.longitude > UL[0]) & (tweets.longitude < BR[0]) & (tweets.latitude < UL[1]) & (tweets.latitude > BR[1])]
    print('len(tweets_clip): ', len(tweets_clip))
    print('ReadTweetsInRectangle finished. ')
    return tweets_clip

 
#cude = np.zeros((rows, cols, 31*24*5*3)).astype('int8')
#cube = np.zeros((rows, cols, layer_cnt), dtype='uint8')

# x = np.random.rand(9000, 9000).astype('uint8')
# print('x.nbytes: {0} bytes'.format(x.nbytes))
# print('sys.getsizeof(x): {0} bytes'.format(sys.getsizeof(x)))
#print(sys.getsizeof(cude))
#print(cude)
#

#np.savez_compressed("D:\\Buston\\numpy_cube.npz")
#print(cube.sum())
#FillCube(cube, tweets_clip, resolu_x, resolu_y, UL[0], UL[1])
def SaveNetCDF4(cube, path_name):
    rootgrp = Dataset(path_name, "w", format="NETCDF3_64BIT_DATA")
    #print('rootgrp.data_model: ', rootgrp.data_model)
    lat_dim = rootgrp.createDimension("lat", rows)
    lon_dim = rootgrp.createDimension("lon", cols)
    time_dim = rootgrp.createDimension("time", layer_cnt)
    print('rootgrp.dimensions: ', rootgrp.dimensions)
    rootgrp.title = 'Boston'
    print(rootgrp.title)

    #print('bins_x,bins_y, bins_z:  ', bins_x,bins_y, bins_z)
    lat = rootgrp.createVariable("lat", 'u2', ('lat', ), zlib=True)
    print('len(lat): ', len(lat))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'

    lon = rootgrp.createVariable("lon", 'u2', ('lon', ), zlib=True)
    time = rootgrp.createVariable("time", 'u2', ('time', ), zlib=True)
    time.units = 'minutes since 2017-01-01'

    lat[:]  = UL[0] + resolu_y * np.arange(rows)
    lon[:]  = BR[1] + resolu_x * np.arange(cols)
    print(lat)
    print(lon)
    #print('bins_x: ', rootgrp.dimensions['bins_x'])
    sum_tw = rootgrp.createVariable('sum_tw', 'u1', ('lat', 'lon', 'time'), zlib=True)  # note: unlimited dimension is leftmost
    print(sum_tw)
    print('cube.shape: ', cube.shape)
    sum_tw[:, :, :] = cube

    print(sum_tw)
    #a =  cube.reshape((cols * rows * layer_cnt))
    #print(a)
    print(cube.shape)
    print(    rootgrp.__dict__)
    for name in rootgrp.ncattrs():
        print(" Global attr: ", name, getattr(rootgrp, name))

    row = 0
    col = 0
    rootgrp.close()

def SplitTwts2Months(df, saved_path):
    months = twts.resample('M').index.strftime('%Y-%m')
    for month in months:
        m = df[month]
        m.to_csv(saved_path + '\\' + month + '.csv')
        print("Processed month: ", month)
        
#SplitTwts2Months(twts, 'D:\\Buston\\Month_csv')

def Add_DayUserCnt2HDF5(srs):
    files = glob.glob('D:\\Buston\\DailyHDF\\*.hdf')
    print("Start to append user count HDF5...")
    for path_name in files:
        with h5py.File(path_name, 'a') as outfile:
            #dset = outfile.create_dataset('S_T_cube', data=cube, chunks=True, compression="gzip")
            #outfile.create_group('Data Fields')
            #dset = outfile.create_dataset('sGroup', data=cube, chunks=True, compression="gzip", shuffle=False)
            #dset = outfile.create_dataset('Data_Fields/User_Cnt', data=userCnt_cube, chunks=(rows, cols, 1), compression="gzip", shuffle=False)
            outfile.attrs['resolu_x'] = resolu_x
            outfile.attrs['resolu_y'] = resolu_y
            outfile.attrs['UL_x'] = UL[0]
            outfile.attrs['UL_y'] = UL[1]
            path_name = os.path.basename(path_name)
            path_name = os.path.splitext(path_name)[0]
            outfile.attrs['Day_user_cnt'] = srs.loc[path_name]['userID']
    print("Append HDF5 finished.")
    

def Add_ConvUserCnt2HDF5(path, ext):
    files = GetAllfiles(path, ext)
    print("Start to append convolution of user count HDF5...")
    #conv = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
    kernel = 7
    conv = np.ones((kernel, kernel))
    print(conv)
    
    for path_name in files:
        
        with h5py.File(path_name, 'a') as outfile:
            cube = np.zeros(outfile['Data_Fields/User_Cnt'].shape, dtype='uint16')
            #print(cube.shape
            layer_cnt = outfile['Data_Fields/User_Cnt'].shape[2]
#            outfile['Data_Fields/User_Cnt_conv7'] = outfile['User_Cnt_conv[[1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]\n [1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]]']
#
            #del outfile['User_Cnt_conv[[1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]\n [1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]\n[1. 1. 1. 1. 1. 1. 1.]]']
            for i in range(layer_cnt):
                #print('Before conv: ', outfile['Data_Fields/User_Cnt'][:, :, i].sum())
                cube[:, :, i] = nd.convolve(outfile['Data_Fields/User_Cnt'][:, :, i], conv)
            outfile.create_dataset('Data_Fields/User_Cnt_conv' + str(kernel), data=cube, chunks=(rows, cols, 1), compression="gzip", shuffle=False)
        print('Finished: ', path_name)
                #print('Current layer: ', i)
                #print('coved.sum: ', conved.sum())
            
            #dset = outfile.create_dataset('S_T_cube', data=cube, chunks=True, compression="gzip")
            #outfile.create_group('Data Fields')
            #dset = outfile.create_dataset('sGroup', data=cube, chunks=True, compression="gzip", shuffle=False)
            #dset = outfile.create_dataset('Data_Fields/User_Cnt', data=userCnt_cube, chunks=(rows, cols, 1), compression="gzip", shuffle=False)
#             outfile.attrs['resolu_x'] = resolu_x
#             outfile.attrs['resolu_y'] = resolu_y
#             outfile.attrs['UL_x'] = UL[0]
#             outfile.attrs['UL_y'] = UL[1]
#             path_name = os.path.basename(path_name)
#             path_name = os.path.splitext(path_name)[0]
#             outfile.attrs['Day_user_cnt'] = srs.loc[path_name]['userID']
    print("Append HDF5 finished.")
    

def CSV2H5(csv, h5):
    df = pd.read_csv(csv)
    df['userVerified']= df['userVerified'].astype(bool)
    df.to_hdf(h5, 'tweets', append=True)
    
#%time CSV2H5(csv_path, "D:\\Buston\\twts.h5")
    
def ReadNetCDF4(path_name):
    print("Start to read netCDF4...")
    f = Dataset(path_name)
    print(f)
    for y in range(f.variables["sum_tw"].shape[1]):
        for x in range(f.variables["sum_tw"].shape[0]):
            t = f.variables["sum_tw"][x, y, :].sum()
            if t > 50:
                print(t)
    print('End. ')



def ReadHDF5(path_name):
    f = h5py.File(path_name, 'r')
    print('len(f.keys()): ', len(f.keys()))
    print('f.keys(): ', f.keys())
    #S_T_cube = f['S_T_cube'][:,:,:]
    #print(f['S_T_cube'].max())
    #cu = f['S_T_cube'][:, :, :]#.astype('uint8')
    quarts = []
    for i in range(f['S_T_cube'].shape[2]):
        #print(f['S_T_cube'][:, :, i].sum())
        quarts.append(f['S_T_cube'][3, 3, i])
        if i % 10 == 0:
            print(i)
            #print(quarts[i])
    #print(cu.sum())
    print(quarts)
    
def GetQuadtree(row):
    return Tile.for_latitude_longitude(row['latitude'], row['longitude'], Quadtree_level).quad_tree

def ReadNumpy(path_name):
    print("Start to read numpy...")
    f = np.fromfile(path_name, dtype='uint8')
    print('f.shape: ', f.shape)
    f.reshape = (rows, cols, layer_cnt)
    print(f.shape)
    for y in range(f.shape[1]):
        for x in range(f.shape[0]):
            t = f[x, y, :].sum()
            if t > 50:
                print(t)
    print('End. ')

def UserCntInBin(cube, df,res_x, res_y, ori_x, ori_y):
    try:
        #print(df.loc[5])
        row_cnt = len(df)
        print('row_cnt: ', row_cnt)
        i = 0

        idx_lon = df.columns.get_loc('longitude')
        idx_lat = df.columns.get_loc('latitude')
        idx_time = df.columns.get_loc('date')
        idx_userID = df.columns.get_loc('userID')

        print("Start to get the user number in each bin...")
        df['row'] = 0
        df['col'] = 0
        df['layer'] = 0
        df['xyz'] = 'abc'

        idx_row = df.columns.get_loc('row')
        idx_col = df.columns.get_loc('col')
        idx_layer = df.columns.get_loc('layer')
        idx_xyz = df.columns.get_loc('xyz')

        while i < row_cnt:
            #print(df.iloc[i])
            #print(df.iat[i, idx_lon])
            #print(df.iat[i, idx_lat])

            row, col = GetRowCol(res_x, res_y, df.iat[i, idx_lon], df.iat[i, idx_lat], ori_x, ori_y)
            #post_date = datetime.datetime.strptime(df.iat[i, idx_time],  '%Y-%m-%d %H:%M:%S')
            post_date = df.iat[i, idx_time]
            
            time_past = ( post_date - start_date)
            layer = int(time_past.total_seconds()/60/time_interval)
            #print(post_date)

            #cube[col, row, layer] +=1
            df.iat[i, idx_row] = row
            df.iat[i, idx_col] = col
            df.iat[i, idx_layer] = layer


#             if i % 100000 == 0:
#                 print('df.iat[i, idx_layer]: ', df.iat[i, idx_layer])
            i = i + 1
        df['xyz'] = df['row'].astype(str) + "," + df["col"].astype(str) + ',' + df["layer"].astype(str)
        userCnts = df.groupby('xyz').size().reset_index(name='counts')
        print("User number in each bin was got. ")
        #print('df[idx_xyz]: ', df[df.xyz != "0,0,0"] )
        #print('userCnts: ', userCnts.columns)
        print('userCnts.counts.sum(): ', userCnts.counts.sum())
        print('len(userCnts): ', len(userCnts))
        #print('userCnts: ', userCnts[userCnts.counts > 1])

        # Fill the cube
        row_cnt = len(userCnts)
        i = 0
        while i < row_cnt:
            row, col, layer = userCnts.iat[i, 0].split(",")
            #print('row, col, layer: ', row, col, layer)
            cube[int(col), int(row), int(layer)] += 1

            i += 1
        #print(cube)
        #cube.tofile('cube_userCnt.bin')
        print("Finished! ")
        #print('userCnts: ', userCnts)
        #return userCnts
        return cube
    except Exception as e:
        print("Get user number  fail error: ", repr(e))
        #print(df.iloc[i])

#%timeit

print('start to read csv file...')
%time twts = ReadTweetsInRectangle(csv_path, UL, BR)
twts['userVerified']= twts['userVerified'].astype(bool)

twts['date'] = pd.to_datetime(twts['date'])
twts = twts.set_index('date')
twts = twts.sort_index()
#print(twts.index.values)
twts['date'] = twts.index

mid_point:  (-71.08505650000001, 42.368171000000004)
328315.99825595925 4692760.795869267
row, col:  20 14
resolu_x, resolu_y:  0.011840883711002448 0.009220345957345444
start to read csv file...


C:\Users\GPU\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


len(tweets_clip):  848017
ReadTweetsInRectangle finished. 
Wall time: 15.1 s


In [46]:
def Fill_Twts_Users_Cnt_Cube(df,res_x, res_y, ori_x, ori_y):
    try:
        layerCnt = int(24*60/time_interval)
        print('24*60/time_interval: ', layerCnt)
        twts_cnt = np.zeros((rows, cols, layerCnt), dtype='uint32')
        users_cnt = np.zeros((rows, cols, layerCnt), dtype='uint32')
        
        print('users_cnt.shape: ', users_cnt.shape)
        
        row_cnt = len(df)
        print('row_cnt: ', row_cnt)
        i = 0

        idx_lon = df.columns.get_loc('longitude')
        idx_lat = df.columns.get_loc('latitude')
        idx_time = df.columns.get_loc('date')
        idx_userID = df.columns.get_loc('userID')

        print("Start to get the user number in each bin...")
        df['row'] = 0
        df['col'] = 0
        df['layer'] = 0
        df['xyz'] = 'abc'

        idx_row = df.columns.get_loc('row')
        idx_col = df.columns.get_loc('col')
        idx_layer = df.columns.get_loc('layer')
        idx_xyz = df.columns.get_loc('xyz')

        while i < row_cnt:
            #print(df.iloc[i])
            #print(df.iat[i, idx_lon])
            #print(df.iat[i, idx_lat])

            row, col = GetRowCol(res_x, res_y, df.iat[i, idx_lon], df.iat[i, idx_lat], ori_x, ori_y)
            #post_date = datetime.datetime.strptime(df.iat[i, idx_time],  '%Y-%m-%d %H:%M:%S')
            post_date = df.iat[i, idx_time]
            #print('post_date: ', post_date)
            
            time_past = ( post_date - start_date)
            layer = int(time_past.total_seconds()/60/time_interval)
            if layer > 23:
                print('post_date: {}, layer = {}'.format(post_date, layer))
            twts_cnt[col, row, layer] +=1
            
            df.iat[i, idx_row] = row
            df.iat[i, idx_col] = col
            df.iat[i, idx_layer] = layer


#             if i % 100000 == 0:
#                 print('df.iat[i, idx_layer]: ', df.iat[i, idx_layer])
            i = i + 1
        df['xyz'] = df['row'].astype(str) + "," + df["col"].astype(str) + ',' + df["layer"].astype(str)
        #userCnts = df.groupby('xyz').size().reset_index(name='counts')
        userCnts = df.groupby('xyz').agg({'userID': pd.Series.nunique})
        print("User number in each bin was got. ")
        #print('df[idx_xyz]: ', df[df.xyz != "0,0,0"] )
        print('userCnts.columns: ', userCnts.columns)
        #print('userCnts: ', userCnts)
        #print('userCnts.counts.sum(): ', userCnts.counts.sum())
        print('len(userCnts): ', len(userCnts))
        #print('userCnts: ', userCnts[userCnts.counts > 1])
        
        day_userCnt = df['userID'].unique()
        print('len of day_userCnt: ', len(day_userCnt))

        # Fill the cube
        row_cnt = len(userCnts)
        i = 0
        while i < row_cnt:
            #print("userCnts.index[i].split(","): ", userCnts.index[i].split(","))
            row, col, layer = userCnts.index[i].split(",")
            #print('row, col, layer: ', row, col, layer)
            users_cnt[int(col), int(row), int(layer)] = userCnts.iloc[i]['userID']  
            i += 1
        #print(cube)
        #cube.tofile('cube_userCnt.bin')
        print("Finished! ")
        #print('userCnts: ', userCnts)
        #return userCnts
        return twts_cnt, users_cnt, day_userCnt
    except Exception as e:
        print("Get user number error: ", repr(e))
        #print(df.iloc[i])    




def ReadHDF5(path_name):
    print(path_name)
    f = h5py.File(path_name, 'r')
    #print('len(f.keys()): ', len(f.keys()))
    #print('f.keys(): ', f.keys())
    #S_T_cube = f['S_T_cube'][:,:,:]
    #print(f['S_T_cube'].max())
    #cu = f['S_T_cube'][:, :, :]#.astype('uint8')
    hours = []
    User_cnt = 0
    #current_layer = 0
    for i in range(f['Data_Fields/User_Cnt'].shape[2]):
        #hours.append(f['Data_Fields/User_Cnt'][:, :, i].sum())
        global User_Cnt, current_layer
        
        #User_cnr_cube[:, :, current_layer] = f['Data_Fields/User_Cnt'][:, :, i]
        #User_cnt = User_cnt + sum(hours)
        #current_layer +=1
        print("Hour: ", i, f['Data_Fields/User_Cnt'][:, :, i].sum())



#print(twts['date'].map(pd.Timestamp.date).unique())

#day_twts = twts.loc['20170101']
#print('len day_twts: ', len(day_twts))

def SaveHDF5(twtsCnt_cube, userCnt_cube, day_userCnt, path_name):
    print("Start to save HDF5...")
    with h5py.File(path_name, 'w') as outfile:
        #dset = outfile.create_dataset('S_T_cube', data=cube, chunks=True, compression="gzip")
        #outfile.create_group('Data Fields')
        dset = outfile.create_dataset('Data_Fields/Tweets_Cnt', data=twtsCnt_cube, chunks=(rows, cols, 1), compression="gzip", shuffle=False)
        dset = outfile.create_dataset('Data_Fields/User_Cnt', data=userCnt_cube, chunks=(rows, cols, 1), compression="gzip", shuffle=False)
        
        dset.attrs['resolu_x'] = resolu_x
        dset.attrs['resolu_y'] = resolu_y
        dset.attrs['UL_x'] = UL[0]
        dset.attrs['UL_y'] = UL[1]
        dset.attrs['day_userCnt'] = day_userCnt
        

        
        # add dimension
        dim_x = np.arange(UL[0], BR[0], resolu_x).tolist()
        dim_y = np.arange(BR[1], UL[1], resolu_y).tolist()
        dim_time = np.arange(0, twtsCnt_cube.shape[2], 1).tolist()
        
        print('dset.shape: ', dset.shape)
        print('len of dim_x: ', len(dim_x))
        print('len of dim_y: ', len(dim_y))
        
        #print(dset[:,:,12].sum())
        #dset.dims[0].label = 'latitude'
       # dset.dims[1].label = 'lonitude'
        outfile['latitude'] = dim_y
        outfile['longitude'] = dim_x
        outfile['time'] = dim_time
        
        outfile['Data_Fields/User_Cnt'].dims[0].label = 'latitude'
        outfile['Data_Fields/User_Cnt'].dims[1].label = 'longitude'
        outfile['Data_Fields/User_Cnt'].dims[2].label = 'time'
        outfile['Data_Fields/User_Cnt'].dims.create_scale(outfile['latitude'])
        outfile['Data_Fields/User_Cnt'].dims.create_scale(outfile['longitude']) 
        outfile['Data_Fields/User_Cnt'].dims.create_scale(outfile['time'])     
        outfile['Data_Fields/User_Cnt'].dims[0].attach_scale(outfile['latitude'])
        outfile['Data_Fields/User_Cnt'].dims[1].attach_scale(outfile['longitude'])
        outfile['Data_Fields/User_Cnt'].dims[2].attach_scale(outfile['time'])
        
        outfile['Data_Fields/Tweets_Cnt'].dims[0].label = 'latitude'
        outfile['Data_Fields/Tweets_Cnt'].dims[1].label = 'longitude'
        outfile['Data_Fields/Tweets_Cnt'].dims[2].label = 'time'
        outfile['Data_Fields/Tweets_Cnt'].dims.create_scale(outfile['latitude'])
        outfile['Data_Fields/Tweets_Cnt'].dims.create_scale(outfile['longitude']) 
        outfile['Data_Fields/Tweets_Cnt'].dims.create_scale(outfile['time'])     
        outfile['Data_Fields/Tweets_Cnt'].dims[0].attach_scale(outfile['latitude'])
        outfile['Data_Fields/Tweets_Cnt'].dims[1].attach_scale(outfile['longitude'])
        outfile['Data_Fields/Tweets_Cnt'].dims[2].attach_scale(outfile['time'])
        
 
        #print(dset.attrs['resolu_y'])
        #dset['Data_Fields/Tweets_Cnt'].dims.create_scale(dim_y)
    print("Save HDF5 finished.\n\n")

def GenerateDailyFiles(df, date_field, path):
    df[date_field] = pd.to_datetime(df[date_field])
    df = df.set_index(date_field)
    df = df.sort_index()
    df['date'] = df.index
    dates = df.resample('D').index.strftime('%Y%m%d')
    print(r'df.resample(D): ', df.resample('D').index)
    #print('dates: ', dates)
    if not os.path.exists(path):
        os.mkdir(path)
    for date in dates:
        global start_date
        start_date = pd.to_datetime(date, format='%Y%m%d')
        twts_eachday = df.loc[date]
        #print('twts_eachday: ', twts_eachday)
        #hours = twts_eachday.resample('H').size()
        #hours = twts_eachday.resample('H').index.hour    
        twts_cube, user_cube, day_userCnt = Fill_Twts_Users_Cnt_Cube(twts_eachday, resolu_x, resolu_y, UL[0], UL[1])
        print(date)
        SaveHDF5( twts_cube, user_cube, day_userCnt, path + '\\' + date + '.hdf')
    
    return df    



In [47]:
#day_twts = twts.loc['2017-01-01']
df = GenerateDailyFiles(twts, 'date', r"D:\Buston\DailyHDF")
#df = GenerateDailyFiles(day_twts, 'date', r"D:\Buston\DailyHDF")

C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:179: FutureWarning: 
.resample() is now a deferred operation
You called index(...) on this deferred object which materialized it into a dataframe
by implicitly taking the mean.  Use .resample(...).mean() instead
C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:180: FutureWarning: 
.resample() is now a deferred operation
You called index(...) on this deferred object which materialized it into a dataframe
by implicitly taking the mean.  Use .resample(...).mean() instead


df.resample(D):  DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2018-03-23', '2018-03-24', '2018-03-25', '2018-03-26',
               '2018-03-27', '2018-03-28', '2018-03-29', '2018-03-30',
               '2018-03-31', '2018-04-01'],
              dtype='datetime64[ns]', name='date', length=456, freq='D')
24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1877
Start to get the user number in each bin...


C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  145
len of day_userCnt:  699
Finished! 
20170101
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1828
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  150
len of day_userCnt:  615
Finished! 
20170102
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1899
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  151
len of day_userCnt:  547
Finished! 
20170103
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*

24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  2386
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  142
len of day_userCnt:  681
Finished! 
20170124
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1278
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  132
len of day_userCnt:  504
Finished! 
20170125
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  2453
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  156
len of day_userCnt:  794
Finished! 
2017

...

User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  121
len of day_userCnt:  470
Finished! 
20171227
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1375
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  114
len of day_userCnt:  569
Finished! 
20171228
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1488
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  129
len of day_userCnt:  614
Finished! 
20171229
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*

24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1462
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  126
len of day_userCnt:  611
Finished! 
20180119
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1651
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  133
len of day_userCnt:  719
Finished! 
20180120
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1429
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  122
len of day_userCnt:  576
Finished! 
2018

User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  143
len of day_userCnt:  650
Finished! 
20180210
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1317
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  124
len of day_userCnt:  627
Finished! 
20180211
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1260
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  130
len of day_userCnt:  556
Finished! 
20180212
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*

24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1260
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  127
len of day_userCnt:  564
Finished! 
20180305
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1225
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  121
len of day_userCnt:  563
Finished! 
20180306
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1127
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  115
len of day_userCnt:  554
Finished! 
2018

User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  123
len of day_userCnt:  566
Finished! 
20180327
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  931
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  111
len of day_userCnt:  552
Finished! 
20180328
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*60/time_interval:  1
users_cnt.shape:  (20, 14, 1)
row_cnt:  1030
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.columns:  Index(['userID'], dtype='object')
len(userCnts):  118
len of day_userCnt:  638
Finished! 
20180329
Start to save HDF5...
dset.shape:  (20, 14, 1)
len of dim_x:  14
len of dim_y:  20
Save HDF5 finished.


24*6

In [5]:
#print(twts.index)
#month_twts = twts.loc['2017-01']

#print(month_twts)
#day_twts['Quadtree'] = ''
#print(day_twts.dtypes)

                                                            
#month_twts['quadtree'] = month_twts.apply(GetQuadtree, axis=1)
#print(month_twts['quadtree'])

#gb = month_twts.groupby(month_twts.quadtree.str[0:13])

#quadtree_cnts = gb.size().reset_index(name='counts')
#print(quadtree_cnts)


C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [56]:
day_twts.to_csv(r"D:\Buston\20170101twts.csv")

C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: 
.resample() is now a deferred operation
You called index(...) on this deferred object which materialized it into a dataframe
by implicitly taking the mean.  Use .resample(...).mean() instead


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1877
Start to get the user number in each bin...


C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

User number in each bin was got. 
userCnts.counts.sum():  1877
len(userCnts):  984
Finished! 
20170101
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1828
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1828
len(userCnts):  989
Finished! 
20170102
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1899
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1899
len(userCnts):  1091
Finished! 
20170103
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  2118
Start to get the user number in each bin...
User numb

User number in each bin was got. 
userCnts.counts.sum():  2453
len(userCnts):  1349
Finished! 
20170126
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  2530
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  2530
len(userCnts):  1455
Finished! 
20170127
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  2644
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  2644
len(userCnts):  1481
Finished! 
20170128
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  2649
Start to get the user number in each bin...
User nu

...

User number in each bin was got. 
userCnts.counts.sum():  1470
len(userCnts):  758
Finished! 
20171217
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1393
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1393
len(userCnts):  748
Finished! 
20171218
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1361
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1361
len(userCnts):  728
Finished! 
20171219
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1408
Start to get the user number in each bin...
User numbe

User number in each bin was got. 
userCnts.counts.sum():  1279
len(userCnts):  694
Finished! 
20180111
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1391
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1391
len(userCnts):  803
Finished! 
20180112
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1338
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1338
len(userCnts):  778
Finished! 
20180113
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1340
Start to get the user number in each bin...
User numbe

User number in each bin was got. 
userCnts.counts.sum():  1244
len(userCnts):  647
Finished! 
20180205
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1207
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1207
len(userCnts):  639
Finished! 
20180206
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1279
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1279
len(userCnts):  709
Finished! 
20180207
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1217
Start to get the user number in each bin...
User numbe

User number in each bin was got. 
userCnts.counts.sum():  1204
len(userCnts):  732
Finished! 
20180302
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1294
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1294
len(userCnts):  768
Finished! 
20180303
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1215
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1215
len(userCnts):  730
Finished! 
20180304
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1260
Start to get the user number in each bin...
User numbe

User number in each bin was got. 
userCnts.counts.sum():  956
len(userCnts):  615
Finished! 
20180327
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  931
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  931
len(userCnts):  604
Finished! 
20180328
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1030
Start to get the user number in each bin...
User number in each bin was got. 
userCnts.counts.sum():  1030
len(userCnts):  676
Finished! 
20180329
Start to save HDF5...
dset.shape:  (975, 665, 24)
len of dim_x:  665
len of dim_y:  975
Save HDF5 finished.


24*60/time_interval:  24
users_cnt.shape:  (975, 665, 24)
row_cnt:  1039
Start to get the user number in each bin...
User number i

In [29]:
# Get dimesion

test_twts = twts.loc['2017-01-01']

print(' UL[0], UL[1]: ',  UL[0], UL[1])
print('resolu_x, resolu_y: ', resolu_x, resolu_y,)
dim_x = np.arange(UL[0], BR[0], resolu_x).tolist()
dim_y = np.arange(BR[1], UL[1], resolu_y).tolist()
#print(dim_x)
print(len(dim_x))
print(len(dim_y))


f = h5py.File(r'D:\Buston\DailyHDF\20170101.hdf', 'r')
[dim.label for dim in f['Data_Fields/Tweets_Cnt'].dims]
for dim in f['Data_Fields/User_Cnt'].dims:
    print(dim.label)
    
print(mid_point[0])
print(mid_point[1])
print(f['Data_Fields/User_Cnt'].dims[0])
    
f.close()

 UL[0], UL[1]:  -71.163736 42.458042
resolu_x, resolu_y:  0.00023678375544022856 0.00018441830410864668
665
975
latitude
longitude
time
-71.08505650000001
42.368171000000004
<"latitude" dimension 0 of HDF5 dataset at 2704788648184>


['', '', '']

D:\Buston\DailyHDF\20170101.hdf
Hour:  0 0
Hour:  1 0
Hour:  2 0
Hour:  3 0
Hour:  4 1
Hour:  5 9
Hour:  6 19
Hour:  7 16
Hour:  8 37
Hour:  9 60
Hour:  10 72
Hour:  11 86
Hour:  12 77
Hour:  13 71
Hour:  14 85
Hour:  15 81
Hour:  16 72
Hour:  17 46
Hour:  18 53
Hour:  19 43
Hour:  20 44
Hour:  21 47
Hour:  22 32
Hour:  23 33
Wall time: 1.4 s
D:\Buston\DailyHDF\20170102.hdf
Hour:  0 17
Hour:  1 13
Hour:  2 10
Hour:  3 9
Hour:  4 4
Hour:  5 10
Hour:  6 9
Hour:  7 34
Hour:  8 38
Hour:  9 57
Hour:  10 51
Hour:  11 53
Hour:  12 65
Hour:  13 65
Hour:  14 83
Hour:  15 81
Hour:  16 65
Hour:  17 72
Hour:  18 45
Hour:  19 71
Hour:  20 41
Hour:  21 26
Hour:  22 25
Hour:  23 45
Wall time: 1.38 s
D:\Buston\DailyHDF\20170103.hdf
Hour:  0 8
Hour:  1 8
Hour:  2 13
Hour:  3 8
Hour:  4 12
Hour:  5 10
Hour:  6 14
Hour:  7 50
Hour:  8 75
Hour:  9 87
Hour:  10 69
Hour:  11 58
Hour:  12 81
Hour:  13 65
Hour:  14 58
Hour:  15 60
Hour:  16 83
Hour:  17 93
Hour:  18 52
Hour:  19 45
Hour:  20 43
Hour:  21 40
H

Hour:  12 58
Hour:  13 69
Hour:  14 57
Hour:  15 85
Hour:  16 85
Hour:  17 142
Hour:  18 68
Hour:  19 73
Hour:  20 56
Hour:  21 51
Hour:  22 37
Hour:  23 28
Wall time: 1.38 s
D:\Buston\DailyHDF\20170125.hdf
Hour:  0 19
Hour:  1 20
Hour:  2 7
Hour:  3 6
Hour:  4 8
Hour:  5 8
Hour:  6 20
Hour:  7 54
Hour:  8 89
Hour:  9 81
Hour:  10 80
Hour:  11 34
Hour:  12 0
Hour:  13 0
Hour:  14 0
Hour:  15 0
Hour:  16 0
Hour:  17 0
Hour:  18 50
Hour:  19 96
Hour:  20 77
Hour:  21 66
Hour:  22 40
Hour:  23 31
Wall time: 1.38 s
D:\Buston\DailyHDF\20170126.hdf
Hour:  0 21
Hour:  1 11
Hour:  2 13
Hour:  3 3
Hour:  4 11
Hour:  5 18
Hour:  6 27
Hour:  7 54
Hour:  8 88
Hour:  9 86
Hour:  10 88
Hour:  11 83
Hour:  12 79
Hour:  13 74
Hour:  14 73
Hour:  15 81
Hour:  16 77
Hour:  17 101
Hour:  18 82
Hour:  19 80
Hour:  20 81
Hour:  21 56
Hour:  22 29
Hour:  23 33
Wall time: 1.36 s
D:\Buston\DailyHDF\20170127.hdf
Hour:  0 32
Hour:  1 13
Hour:  2 15
Hour:  3 13
Hour:  4 10
Hour:  5 17
Hour:  6 29
Hour:  7 56
Hou

Hour:  21 64
Hour:  22 48
Hour:  23 45
Wall time: 1.38 s
D:\Buston\DailyHDF\20170217.hdf
Hour:  0 38
Hour:  1 22
Hour:  2 21
Hour:  3 12
Hour:  4 7
Hour:  5 10
Hour:  6 49
Hour:  7 61
Hour:  8 127
Hour:  9 87
Hour:  10 66
Hour:  11 87
Hour:  12 84
Hour:  13 83
Hour:  14 63
Hour:  15 78
Hour:  16 187
Hour:  17 206
Hour:  18 86
Hour:  19 80
Hour:  20 63
Hour:  21 56
Hour:  22 47
Hour:  23 31
Wall time: 1.4 s
D:\Buston\DailyHDF\20170218.hdf
Hour:  0 24
Hour:  1 14
Hour:  2 10
Hour:  3 13
Hour:  4 4
Hour:  5 10
Hour:  6 19
Hour:  7 30
Hour:  8 108
Hour:  9 90
Hour:  10 122
Hour:  11 109
Hour:  12 120
Hour:  13 103
Hour:  14 114
Hour:  15 100
Hour:  16 86
Hour:  17 106
Hour:  18 79
Hour:  19 75
Hour:  20 70
Hour:  21 61
Hour:  22 59
Hour:  23 48
Wall time: 1.36 s
D:\Buston\DailyHDF\20170219.hdf
Hour:  0 45
Hour:  1 15
Hour:  2 18
Hour:  3 18
Hour:  4 8
Hour:  5 9
Hour:  6 17
Hour:  7 24
Hour:  8 71
Hour:  9 62
Hour:  10 81
Hour:  11 97
Hour:  12 102
Hour:  13 129
Hour:  14 97
Hour:  15 110


Hour:  2 0
Hour:  3 12
Hour:  4 7
Hour:  5 7
Hour:  6 15
Hour:  7 29
Hour:  8 49
Hour:  9 62
Hour:  10 65
Hour:  11 97
Hour:  12 105
Hour:  13 102
Hour:  14 91
Hour:  15 100
Hour:  16 103
Hour:  17 78
Hour:  18 90
Hour:  19 69
Hour:  20 69
Hour:  21 61
Hour:  22 61
Hour:  23 48
Wall time: 1.38 s
D:\Buston\DailyHDF\20170313.hdf
Hour:  0 43
Hour:  1 27
Hour:  2 18
Hour:  3 12
Hour:  4 9
Hour:  5 14
Hour:  6 26
Hour:  7 44
Hour:  8 81
Hour:  9 81
Hour:  10 77
Hour:  11 90
Hour:  12 76
Hour:  13 89
Hour:  14 90
Hour:  15 85
Hour:  16 127
Hour:  17 108
Hour:  18 96
Hour:  19 67
Hour:  20 74
Hour:  21 53
Hour:  22 62
Hour:  23 42
Wall time: 1.37 s
D:\Buston\DailyHDF\20170314.hdf
Hour:  0 24
Hour:  1 17
Hour:  2 23
Hour:  3 11
Hour:  4 11
Hour:  5 14
Hour:  6 19
Hour:  7 26
Hour:  8 58
Hour:  9 93
Hour:  10 72
Hour:  11 86
Hour:  12 101
Hour:  13 88
Hour:  14 85
Hour:  15 88
Hour:  16 75
Hour:  17 82
Hour:  18 65
Hour:  19 56
Hour:  20 50
Hour:  21 43
Hour:  22 21
Hour:  23 31
Wall time: 1.39

Hour:  8 89
Hour:  9 80
Hour:  10 93
Hour:  11 84
Hour:  12 82
Hour:  13 95
Hour:  14 65
Hour:  15 70
Hour:  16 71
Hour:  17 110
Hour:  18 81
Hour:  19 52
Hour:  20 82
Hour:  21 59
Hour:  22 27
Hour:  23 35
Wall time: 1.38 s
D:\Buston\DailyHDF\20170405.hdf
Hour:  0 19
Hour:  1 18
Hour:  2 8
Hour:  3 7
Hour:  4 6
Hour:  5 9
Hour:  6 29
Hour:  7 45
Hour:  8 98
Hour:  9 77
Hour:  10 93
Hour:  11 91
Hour:  12 102
Hour:  13 93
Hour:  14 79
Hour:  15 79
Hour:  16 86
Hour:  17 91
Hour:  18 76
Hour:  19 68
Hour:  20 66
Hour:  21 58
Hour:  22 47
Hour:  23 38
Wall time: 1.38 s
D:\Buston\DailyHDF\20170406.hdf
Hour:  0 16
Hour:  1 12
Hour:  2 8
Hour:  3 9
Hour:  4 10
Hour:  5 10
Hour:  6 28
Hour:  7 50
Hour:  8 80
Hour:  9 91
Hour:  10 91
Hour:  11 95
Hour:  12 114
Hour:  13 81
Hour:  14 95
Hour:  15 77
Hour:  16 76
Hour:  17 99
Hour:  18 74
Hour:  19 67
Hour:  20 65
Hour:  21 61
Hour:  22 37
Hour:  23 38
Wall time: 1.35 s
D:\Buston\DailyHDF\20170407.hdf
Hour:  0 21
Hour:  1 13
Hour:  2 6
Hour:  3

...

Hour:  19 43
Hour:  20 35
Hour:  21 23
Hour:  22 28
Hour:  23 19
Wall time: 1.37 s
D:\Buston\DailyHDF\20171220.hdf
Hour:  0 19
Hour:  1 9
Hour:  2 6
Hour:  3 5
Hour:  4 6
Hour:  5 7
Hour:  6 18
Hour:  7 37
Hour:  8 45
Hour:  9 39
Hour:  10 42
Hour:  11 48
Hour:  12 37
Hour:  13 47
Hour:  14 51
Hour:  15 40
Hour:  16 56
Hour:  17 62
Hour:  18 41
Hour:  19 35
Hour:  20 40
Hour:  21 32
Hour:  22 26
Hour:  23 25
Wall time: 1.41 s
D:\Buston\DailyHDF\20171221.hdf
Hour:  0 21
Hour:  1 13
Hour:  2 4
Hour:  3 7
Hour:  4 2
Hour:  5 5
Hour:  6 12
Hour:  7 22
Hour:  8 42
Hour:  9 49
Hour:  10 46
Hour:  11 45
Hour:  12 39
Hour:  13 59
Hour:  14 51
Hour:  15 56
Hour:  16 49
Hour:  17 68
Hour:  18 45
Hour:  19 42
Hour:  20 39
Hour:  21 32
Hour:  22 28
Hour:  23 20
Wall time: 1.39 s
D:\Buston\DailyHDF\20171222.hdf
Hour:  0 20
Hour:  1 8
Hour:  2 2
Hour:  3 5
Hour:  4 4
Hour:  5 8
Hour:  6 12
Hour:  7 30
Hour:  8 44
Hour:  9 48
Hour:  10 34
Hour:  11 46
Hour:  12 48
Hour:  13 51
Hour:  14 64
Hour:  15 

Hour:  12 50
Hour:  13 52
Hour:  14 50
Hour:  15 49
Hour:  16 75
Hour:  17 67
Hour:  18 37
Hour:  19 50
Hour:  20 27
Hour:  21 40
Hour:  22 18
Hour:  23 21
Wall time: 1.35 s
D:\Buston\DailyHDF\20180113.hdf
Hour:  0 14
Hour:  1 4
Hour:  2 6
Hour:  3 3
Hour:  4 5
Hour:  5 4
Hour:  6 6
Hour:  7 16
Hour:  8 43
Hour:  9 42
Hour:  10 49
Hour:  11 49
Hour:  12 59
Hour:  13 63
Hour:  14 53
Hour:  15 53
Hour:  16 54
Hour:  17 59
Hour:  18 43
Hour:  19 33
Hour:  20 33
Hour:  21 37
Hour:  22 32
Hour:  23 18
Wall time: 1.37 s
D:\Buston\DailyHDF\20180114.hdf
Hour:  0 12
Hour:  1 9
Hour:  2 3
Hour:  3 4
Hour:  4 4
Hour:  5 5
Hour:  6 7
Hour:  7 20
Hour:  8 20
Hour:  9 40
Hour:  10 40
Hour:  11 38
Hour:  12 48
Hour:  13 43
Hour:  14 49
Hour:  15 48
Hour:  16 59
Hour:  17 69
Hour:  18 32
Hour:  19 35
Hour:  20 37
Hour:  21 27
Hour:  22 23
Hour:  23 16
Wall time: 1.38 s
D:\Buston\DailyHDF\20180115.hdf
Hour:  0 11
Hour:  1 9
Hour:  2 6
Hour:  3 3
Hour:  4 4
Hour:  5 3
Hour:  6 10
Hour:  7 22
Hour:  8 37

Hour:  3 5
Hour:  4 3
Hour:  5 5
Hour:  6 11
Hour:  7 19
Hour:  8 39
Hour:  9 26
Hour:  10 43
Hour:  11 40
Hour:  12 35
Hour:  13 42
Hour:  14 47
Hour:  15 39
Hour:  16 46
Hour:  17 46
Hour:  18 34
Hour:  19 38
Hour:  20 38
Hour:  21 25
Hour:  22 22
Hour:  23 21
Wall time: 1.38 s
D:\Buston\DailyHDF\20180206.hdf
Hour:  0 6
Hour:  1 4
Hour:  2 4
Hour:  3 2
Hour:  4 5
Hour:  5 8
Hour:  6 9
Hour:  7 23
Hour:  8 42
Hour:  9 35
Hour:  10 44
Hour:  11 32
Hour:  12 34
Hour:  13 47
Hour:  14 44
Hour:  15 43
Hour:  16 49
Hour:  17 44
Hour:  18 32
Hour:  19 33
Hour:  20 39
Hour:  21 32
Hour:  22 12
Hour:  23 16
Wall time: 1.38 s
D:\Buston\DailyHDF\20180207.hdf
Hour:  0 10
Hour:  1 6
Hour:  2 12
Hour:  3 5
Hour:  4 10
Hour:  5 5
Hour:  6 12
Hour:  7 21
Hour:  8 46
Hour:  9 44
Hour:  10 41
Hour:  11 31
Hour:  12 51
Hour:  13 56
Hour:  14 52
Hour:  15 39
Hour:  16 51
Hour:  17 40
Hour:  18 39
Hour:  19 29
Hour:  20 33
Hour:  21 36
Hour:  22 19
Hour:  23 21
Wall time: 1.4 s
D:\Buston\DailyHDF\2018020

Hour:  22 20
Hour:  23 21
Wall time: 1.36 s
D:\Buston\DailyHDF\20180301.hdf
Hour:  0 9
Hour:  1 5
Hour:  2 4
Hour:  3 5
Hour:  4 3
Hour:  5 8
Hour:  6 21
Hour:  7 37
Hour:  8 39
Hour:  9 43
Hour:  10 46
Hour:  11 44
Hour:  12 49
Hour:  13 50
Hour:  14 38
Hour:  15 47
Hour:  16 51
Hour:  17 53
Hour:  18 55
Hour:  19 50
Hour:  20 43
Hour:  21 27
Hour:  22 32
Hour:  23 22
Wall time: 1.36 s
D:\Buston\DailyHDF\20180302.hdf
Hour:  0 15
Hour:  1 12
Hour:  2 5
Hour:  3 2
Hour:  4 3
Hour:  5 10
Hour:  6 17
Hour:  7 21
Hour:  8 33
Hour:  9 35
Hour:  10 51
Hour:  11 66
Hour:  12 62
Hour:  13 45
Hour:  14 42
Hour:  15 41
Hour:  16 48
Hour:  17 52
Hour:  18 46
Hour:  19 40
Hour:  20 28
Hour:  21 27
Hour:  22 16
Hour:  23 15
Wall time: 1.39 s
D:\Buston\DailyHDF\20180303.hdf
Hour:  0 7
Hour:  1 11
Hour:  2 7
Hour:  3 6
Hour:  4 3
Hour:  5 5
Hour:  6 6
Hour:  7 20
Hour:  8 19
Hour:  9 40
Hour:  10 51
Hour:  11 44
Hour:  12 45
Hour:  13 66
Hour:  14 56
Hour:  15 59
Hour:  16 55
Hour:  17 59
Hour:  18 5

Hour:  13 48
Hour:  14 47
Hour:  15 69
Hour:  16 46
Hour:  17 44
Hour:  18 36
Hour:  19 42
Hour:  20 43
Hour:  21 37
Hour:  22 25
Hour:  23 26
Wall time: 1.36 s
D:\Buston\DailyHDF\20180325.hdf
Hour:  0 26
Hour:  1 21
Hour:  2 20
Hour:  3 14
Hour:  4 3
Hour:  5 3
Hour:  6 7
Hour:  7 11
Hour:  8 21
Hour:  9 33
Hour:  10 46
Hour:  11 52
Hour:  12 51
Hour:  13 52
Hour:  14 53
Hour:  15 55
Hour:  16 56
Hour:  17 44
Hour:  18 43
Hour:  19 35
Hour:  20 34
Hour:  21 31
Hour:  22 29
Hour:  23 20
Wall time: 1.37 s
D:\Buston\DailyHDF\20180326.hdf
Hour:  0 6
Hour:  1 7
Hour:  2 4
Hour:  3 1
Hour:  4 4
Hour:  5 5
Hour:  6 6
Hour:  7 26
Hour:  8 30
Hour:  9 30
Hour:  10 30
Hour:  11 38
Hour:  12 46
Hour:  13 34
Hour:  14 40
Hour:  15 42
Hour:  16 36
Hour:  17 37
Hour:  18 34
Hour:  19 37
Hour:  20 30
Hour:  21 23
Hour:  22 32
Hour:  23 20
Wall time: 1.36 s
D:\Buston\DailyHDF\20180327.hdf
Hour:  0 10
Hour:  1 4
Hour:  2 3
Hour:  3 3
Hour:  4 2
Hour:  5 2
Hour:  6 20
Hour:  7 27
Hour:  8 34
Hour:  9 3